In [1]:
#Simpleks maks
import numpy as np
from numpy.linalg import inv
from itertools import combinations


def simpleksFirst(C, A, b, base):
  l = len(base)+1

  pivot_e = np.delete(range(len(C)), base)#indeksi ne baznih elemenata

  min_i = np.argmin(-C[pivot_e])#indeks najmanjeg elementa

  if(-C[pivot_e[min_i]] >= 0):
    return M, base  
    
  pivot_row = np.argmin(b/A[:,min_i])#indeks vrste najmanjeg elementa pivot kolone

  #postavka matrice, spajanje 4 dela, pivot kolonu ostvarujemo posebno
  mat = np.empty((l, l))
  mat[0:1,0:l-1] = np.zeros(l-1)
  mat[1:,0:l-1] = np.eye(l-1)
  mat[0:1,l-1:]= 0 #cbb
  mat[1:,l-1:] = b

  pivot = np.empty(l) #pivot kolona
  pivot[0] = -C[pivot_e[min_i]] 
  pivot[1:] = np.reshape(A[:,min_i],(l-1))

  #transformacija matrice
  new = np.array(mat)
  for i in range(l):
    for j in range(l):
      if(i == pivot_row+1):
        new[i,j] = mat[i,j] / pivot[i]
      else:
        new[i,j] = mat[i,j] - mat[pivot_row +1, j]*pivot[i]/pivot[pivot_row+1]
  #imamo novi base
  base[pivot_row] = min_i
  return simpleksStep(C,A,b,new,base)
  

def simpleksStep(C,A,b,M,base):
  w = M[0:1,0:len(base)]

  pivot_e = np.delete(range(len(C)), base)#indeksi ne baznih elemenata

  min = np.matmul(w, A[:,pivot_e[0]])-C[pivot_e[0]] #pretpostvka da je prvi w*aj-cj
  min_i = pivot_e[0]#index elem

  for val in pivot_e[1:]:#iteriramo da li ima bolji kandidat
    if(np.matmul(w, A[:,val])-C[val] < min):
      min = np.matmul(w, A[:,val])-C[val]
      min_i = val

  if(min >= 0):
    return M, base

  pivot_row = np.argmin(np.reshape(M[1:,-1],(A.shape[0],1)) / np.matmul(M[1:,:-1],A[:,min_i])) #pivot vrsta

  pivot = np.empty(A.shape[0]+1) #pivot kolona
  pivot[0] = min
  pivot[1:] = np.reshape(np.matmul(M[1:,:-1],A[:,min_i]),(A.shape[0]))
  
  new = np.array(M)
  for i in range(A.shape[0]+1):
    for j in range(A.shape[0]+1):
      if(i == pivot_row+1):
        new[i,j] = M[i,j] / pivot[i]
      else:
        new[i,j] = M[i,j] - M[pivot_row +1, j]*pivot[i]/pivot[pivot_row+1]
  base[pivot_row] = min_i
  return simpleksStep(C,A,b,new,base)

def simpleks(C, A, b):
  for i in reversed(list(combinations(range(len(C)), A.shape[0]))):
    try:
      if (inv(A[:,i]) * b).min() >= 0:
        M, res_base = simpleksFirst(C, A, b, list(i))
        print("###>> Simplex done <<###")
        print("Max:")
        print(M[0,-1])
        for i, val in enumerate(res_base):
          print(f"Base variable {val} has coeff {M[i+1,-1]}")
        return M[0,-1] ,M, res_base
    except:
      print(f"{i} not a good base")
  print("Non simplexable")




#TEST

C = np.array([6, 14, 13, 0, 0])
A = np.matrix([[0.5, 2, 1, 1, 0],[1, 2, 4, 0, 1]])
b = np.array([[24],[60]])
simpleks(C, A, b)

C = np.array([2, 1.5, 0, 0])
A = np.matrix([[6, 3, 1, 0],[75, 100, 0, 1]])
b = np.array([[1200],[25000]])
simpleks(C, A, b)

C = np.array([1, 2, 3, 0, 0, 0])
A = np.matrix([[1, 2, 3, 1, 0, 0],[2, 3, 1, 0, 1, 0],[4, 5, 6, 0, 0, 1]])
b = np.array([[100],[200],[250]])
a = simpleks(C, A, b)


###>> Simplex done <<###
Max:
294.0
Base variable 0 has coeff 36.0
Base variable 2 has coeff 6.0
###>> Simplex done <<###
Max:
480.0
Base variable 0 has coeff 120.0
Base variable 1 has coeff 160.0
###>> Simplex done <<###
Max:
100.0
Base variable 2 has coeff 33.333333333333336
Base variable 4 has coeff 166.66666666666666
Base variable 5 has coeff 50.0
